In [2]:
import pandas as pd
archivo = './VEM_2019.csv'
V = pd.read_csv(archivo, index_col=0)
V.head()

,id_hogar,id_persona,id_viaje,fecha,lugar_origen,zat_origen,p17_Id_motivo_viaje,hora_inicio_viaje,p28_lugar_destino,zat_destino,...,utam_origen,utam_destino,modo_principal,estrato,tipo_vivienda,propiedad_vivienda,edad,nivel_educativo,ocupación,sexo
0,33754,2,1,43669.0,1.0,204.0,5,0.333333,2.0,554.0,...,UTAM31,UTAM86,SITP Provisional,3,1,3,37,2,38,Mujer
1,33770,2,3,43669.0,2.0,299.0,6,0.625000,1.0,973.0,...,UTAM105,UTAM54,Transporte informal,2,2,1,60,2,31,Mujer
2,3882,3,1,43605.0,1.0,551.0,1,0.250000,2.0,324.0,...,UTAM84,UTAM112,SITP Zonal,2,2,1,28,13,16,Hombre
3,5189,4,1,43554.0,1.0,17.0,1,0.333333,2.0,7.0,...,UTAM10,UTAM1,SITP Zonal,4,1,3,21,10,2,Hombre
4,5636,2,2,43558.0,2.0,554.0,6,0.708333,1.0,933.0,...,UTAM86,UTAM573,Transporte informal,0,2,2,34,7,19,Mujer


In [3]:
V.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134497 entries, 0 to 134496
Data columns (total 37 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id_hogar                           134497 non-null  object 
 1   id_persona                         134497 non-null  int64  
 2   id_viaje                           134497 non-null  int64  
 3   fecha                              127364 non-null  float64
 4   lugar_origen                       127364 non-null  float64
 5   zat_origen                         127363 non-null  float64
 6   p17_Id_motivo_viaje                134497 non-null  int64  
 7   hora_inicio_viaje                  134497 non-null  float64
 8   p28_lugar_destino                  127364 non-null  float64
 9   zat_destino                        127361 non-null  float64
 10  p30_camino_cuadras                 127347 non-null  float64
 11  p30_camino_minutos                 1273

In [4]:
print('Número de atributos iniciales = %d' % (V.shape[1]))
#Añadir una nueva variable: tiempo de viaje en minutos
V['tiempo_viaje'] = (V['p31_hora_llegada'] - V['hora_inicio_viaje']) *(24*60)
#Eliminar atributos de identificación, la fecha y las zonas geográficas
V = V.drop(columns=['id_hogar','id_persona','id_viaje', 'fecha','zat_origen', 'zat_destino', 'utam_origen', 'utam_destino'])
print('Número de atributos posterior = %d' % (V.shape[1]))

Número de atributos iniciales = 37
Número de atributos posterior = 30


In [5]:
#Insertar ceros en las variables binarias
col_binarias = ['p32_lunes','p32_martes','p32_miercoles','p32_jueves','p32_viernes','p32_sabado','p32_domingo','p32_ocasional']
for col in col_binarias:
   V[col] = V[col].fillna(0)
#Insertar valor 'Ninguna' en las variables relacionadas a la aplicación usada
col_app = ['p33_cual_aplicacion_antes_viaje', 'p34_cual_aplicacion_durante_viaje']
for col in col_app:
   V[col] = V[col].fillna('Ninguna')
#Valores perdidos
print('Número de valores perdidos:')
for col in V.columns:
    print('\t%s: %d' % (col,V[col].isna().sum()))

Número de valores perdidos:
	lugar_origen: 7133
	p17_Id_motivo_viaje: 0
	hora_inicio_viaje: 0
	p28_lugar_destino: 7133
	p30_camino_cuadras: 7150
	p30_camino_minutos: 7133
	p31_hora_llegada: 0
	p32_lunes: 0
	p32_martes: 0
	p32_miercoles: 0
	p32_jueves: 0
	p32_viernes: 0
	p32_sabado: 0
	p32_domingo: 0
	p32_ocasional: 0
	p33_aplicacion_antes_viaje: 7133
	p33_cual_aplicacion_antes_viaje: 0
	p34_aplicacion_durante_viaje: 7133
	p34_cual_aplicacion_durante_viaje: 0
	mun_origen: 7281
	mun_destino: 7231
	modo_principal: 0
	estrato: 0
	tipo_vivienda: 0
	propiedad_vivienda: 0
	edad: 0
	nivel_educativo: 0
	ocupación: 0
	sexo: 0
	tiempo_viaje: 0


In [6]:
#Estimar el número de cuadras caminadas para los valores perdidos de acuerdo a la media de las personas que caminaron el mismo número de minutos
import math
import numpy as np
for i in range(V.shape[0]):
  if math.isnan(V['p30_camino_cuadras'][i]) and not math.isnan(V['p30_camino_minutos'][i]):
    a = V['p30_camino_minutos'][i]
    V['p30_camino_cuadras'][i] = V[V['p30_camino_minutos']==a]['p30_camino_cuadras'].mean()
for i in range(V.shape[0]): #Registrar valores 'sin información' como datos faltantes
  if V['mun_destino'][i] == 0: V['mun_destino'][i] = np.NaN
  if V['nivel_educativo'][i] == 99: V['nivel_educativo'][i] =np.NaN
  if V['p17_Id_motivo_viaje'][i] == 99: V['p17_Id_motivo_viaje'][i] = np.NaN
  if V['propiedad_vivienda'][i] == 8: V['p17_Id_motivo_viaje'][i] = np.NaN
for i in range(V.shape[0]): #Transformando el atributo relacionado al uso de aplicación a valores 0 y 1
  if V['p33_aplicacion_antes_viaje'][i] == 2 or V['p33_aplicacion_antes_viaje'][i] == 9 : V['p33_aplicacion_antes_viaje'][i] = 0
#Eliminar los registros con varios datos faltantes
print('\nNúmero de registros iniciales = %d' % (V.shape[0]))
V = V.dropna()
print('Número de registros después de descartar aquellos con datos faltantes = %d' % (V.shape[0]))

/tmp/ipykernel_447264/3120172258.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  V['p30_camino_cuadras'][i] = V[V['p30_camino_minutos']==a]['p30_camino_cuadras'].mean()
/tmp/ipykernel_447264/3120172258.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if V['mun_destino'][i] == 0: V['mun_destino'][i] = np.NaN
/tmp/ipykernel_447264/3120172258.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if V['propiedad_vivienda'][i] == 8: V


Número de registros iniciales = 134497
Número de registros después de descartar aquellos con datos faltantes = 126381


In [7]:
#Registros duplicados
duplicados = pd.DataFrame(V.duplicated(), columns=['B'])
print('Número de registros duplicados = %d' % (duplicados.sum()))

Número de registros duplicados = 119


In [8]:
V = V.drop(columns=['p28_lugar_destino','p33_cual_aplicacion_antes_viaje','p34_aplicacion_durante_viaje','p34_cual_aplicacion_durante_viaje'])
print('Número de atributos final = %d' % (V.shape[1]))

Número de atributos final = 26


In [9]:
V

,lugar_origen,p17_Id_motivo_viaje,hora_inicio_viaje,p30_camino_cuadras,p30_camino_minutos,p31_hora_llegada,p32_lunes,p32_martes,p32_miercoles,p32_jueves,...,mun_destino,modo_principal,estrato,tipo_vivienda,propiedad_vivienda,edad,nivel_educativo,ocupación,sexo,tiempo_viaje
0,1.0,5.0,0.333333,5.0,5.0,0.354167,1.0,0.0,0.0,0.0,...,11001.0,SITP Provisional,3,1,3,37,2.0,38,Mujer,30.000001
1,2.0,6.0,0.625000,2.0,3.0,0.666667,0.0,0.0,0.0,0.0,...,11001.0,Transporte informal,2,2,1,60,2.0,31,Mujer,60.000000
2,1.0,1.0,0.250000,4.0,15.0,0.312500,1.0,1.0,1.0,1.0,...,11001.0,SITP Zonal,2,2,1,28,13.0,16,Hombre,90.000000
3,1.0,1.0,0.333333,5.0,5.0,0.347222,1.0,1.0,1.0,1.0,...,11001.0,SITP Zonal,4,1,3,21,10.0,2,Hombre,20.000000
4,2.0,6.0,0.708333,2.0,3.0,0.770833,1.0,1.0,1.0,1.0,...,25754.0,Transporte informal,0,2,2,34,7.0,19,Mujer,90.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134492,2.0,6.0,0.562500,3.0,5.0,0.645833,1.0,1.0,1.0,1.0,...,25260.0,TransMilenio,2,2,4,30,9.0,16,Mujer,120.000000
134493,2.0,6.0,0.635417,15.0,7.0,0.642361,1.0,1.0,1.0,1.0,...,11001.0,A pie,3,2,3,23,11.0,16,Hombre,9.999999
134494,1.0,1.0,0.645833,7.0,10.0,0.743056,1.0,1.0,1.0,1.0,...,11001.0,TransMilenio,1,1,1,34,7.0,16,Hombre,140.000001
134495,2.0,6.0,0.583333,10.0,15.0,0.593750,1.0,1.0,1.0,1.0,...,25799.0,A pie,2,2,3,24,11.0,17,Mujer,15.000000


In [10]:
# Discretizar camino_cuadras
V['p30_camino_cuadras'] = V['p30_camino_cuadras'].astype(int)
print(V['p30_camino_cuadras'][:3])

# Discretizar camino_minutos 
V['p30_camino_minutos'] = V['p30_camino_minutos'].astype(int)
print(V['p30_camino_minutos'][:3])

# Discretizar hora_llegada -> formato 24h, aproxima a la hora mas cercana
V['p31_hora_llegada'] = V['p31_hora_llegada'].map(lambda i: round(24*i))
print(V['p31_hora_llegada'][:3])

# Discretizar hora_inicio_viaje -> formato 24h, aproxima a la hora mas cercana
V['hora_inicio_viaje'] = V['hora_inicio_viaje'].map(lambda i: round(24*i))
print(V['hora_inicio_viaje'][:3])



0    5
1    2
2    4
Name: p30_camino_cuadras, dtype: int64
0     5
1     3
2    15
Name: p30_camino_minutos, dtype: int64
0     9
1    16
2     8
Name: p31_hora_llegada, dtype: int64
0     8
1    15
2     6
Name: hora_inicio_viaje, dtype: int64


In [ ]:
# One hot encoding mun_origen

V['mun_origen'] = V['mun_origen'].map(lambda i: 'bogota_origen' if i == 11001.0 else 'fuera_bogota_origen')
origen_hot = pd.get_dummies(V['mun_origen'])
print(origen_hot[:3])

In [ ]:
# Once hot encoding mun_destino

V['mun_destino'] = V['mun_destino'].map(lambda i: 'bogota_dest' if i == 11001.0 else 'fuera_bogota_dest')
dest_hot = pd.get_dummies(V['mun_destino'])
print(dest_hot[:5])

In [ ]:
# One hot encoding nivel_educativo

nivel_edu_hot = pd.get_dummies(V['nivel_educativo'])
nivel_edu_hot = nivel_edu_hot.rename(columns={1.0:'preescolar', 2.0:'primaria_inc', 3.0:'primaria_comp', 4.0:'secundaria_inc',\
     5.0:'secundaria_comp', 6.0:'media_inc', 7.0:'media_comp', 8.0:'tecnico_inc', 9.0:'tecnico_comp', 10.0:'univer_inc',\
         11.0:'univer_comp', 12.0:'posg_inc', 13.0:'posg_comp', 14.0:'ninguno'})
print(nivel_edu_hot[:3])

In [ ]:
# One hot encoding propiedad_vivienda

prop_vivienda_hot = pd.get_dummies(V['propiedad_vivienda'])
prop_vivienda_hot = prop_vivienda_hot.rename(columns={1:'propia_pagada', 2:'propia_pagando', 3:'arriendo',\
    4:'subarriendo', 5:'en usufructo', 6:'ocupande_de_hecho', 7:'agregado'})
print(prop_vivienda_hot[:3])

In [ ]:
# One hot encoding tipo_vivienda

tipo_vivienda_hot = pd.get_dummies(V['tipo_vivienda'])
tipo_vivienda_hot = prop_vivienda_hot.rename(columns={1:'casa', 2:'apartamento', 3:'cuarto_inq', 4:'cuarto_otro',\
     5:'vivenda_indigena', 6:'otro_tipo'})
print(prop_vivienda_hot[:3])

        propia_pagada  propia_pagando  arriendo  subarriendo  en usufructo  \
0                   0               0         1            0             0   
1                   1               0         0            0             0   
2                   1               0         0            0             0   
3                   0               0         1            0             0   
4                   0               1         0            0             0   
...               ...             ...       ...          ...           ...   
134492              0               0         0            1             0   
134493              0               0         1            0             0   
134494              1               0         0            0             0   
134495              0               0         1            0             0   
134496              1               0         0            0             0   

        ocupande_de_hecho  agregado  
0                       0

In [12]:
"""
convirtiendo mun_origen y mun_salida cada una en dos variables binarias (0 y 1)
 y después aplicando One-hot encoding para que todo el conjunto quede sólo con variables de tipo binario? Y yo trabajo en el documento

# mun_origen 1 if 11011 else 0
V['mun_origen'] = V['mun_origen'].map(lambda i: 1 if i == 11001.0 else 0)
print(V['mun_origen'][:3])
# mun_destino
print(V['mun_destino'][:3])
V['mun_origen'] = V['mun_origen'].map(lambda i: 1 if i == 11001.0 else 0)
"""

"\nconvirtiendo mun_origen y mun_salida cada una en dos variables binarias (0 y 1)\n y después aplicando One-hot encoding para que todo el conjunto quede sólo con variables de tipo binario? Y yo trabajo en el documento\n\n# mun_origen 1 if 11011 else 0\nV['mun_origen'] = V['mun_origen'].map(lambda i: 1 if i == 11001.0 else 0)\nprint(V['mun_origen'][:3])\n# mun_destino\nprint(V['mun_destino'][:3])\nV['mun_origen'] = V['mun_origen'].map(lambda i: 1 if i == 11001.0 else 0)\n"